In [1]:
import numpy as np
from glob import glob
from os.path import join
import sys
import sh
import os
import math
from slurmpy import Slurm

def read_cmt(file_cmt):
    cmtfile = open(file_cmt,'r')
    for icmt in cmtfile.readlines():
        cmtline = icmt.strip().split()
        if(cmtline[0] == "depth:"):
            dep = float(cmtline[1])
        elif(cmtline[0] == "latitude:"):
            lat = float(cmtline[-1])
        elif(cmtline[0] == "longitude:"):
            lon = float(cmtline[-1])
        elif(cmtline[0] == "Mrr:"):
            Mrr = float(cmtline[-1])
        elif(cmtline[0] == "Mtt:"):
            Mtt = float(cmtline[-1])
        elif(cmtline[0] == "Mpp:"):
            Mpp = float(cmtline[-1])
        elif(cmtline[0] == "Mrt:"):
            Mrt = float(cmtline[-1])
        elif(cmtline[0] == "Mrp:"):
            Mrp = float(cmtline[-1])
        elif(cmtline[0] == "Mtp:"):
            Mtp = float(cmtline[-1])
    return dep,lon,lat,Mrr,Mtt,Mpp,Mrt,Mrp,Mtp

def cmtsolution_per(file_cmt,type_per,out_dir,depth_range=None):
    """
    perturb CMTSOLUTION in depth
    returns the new CMTSOLUTION files in the out_dir
    also returns the depth perturb par list 
    """
    if(not os.path.exists(out_dir)):
        os.mkdir(out_dir)
    if type_per == 'depth':
        #cmtfile=open(file_cmt,'r')
        if(depth_range == None):         # default depth perturbation
            depthrange = ['d12','d8','d4','00','u4','u8','u12','u16']
        else:
            depthrange = depth_range
        for idep in range(0,len(depthrange)):
            cmt_per = open(out_dir+"/"+file_cmt+"_"+depthrange[idep],'w')
            cmtfile = open(file_cmt,'r')
            for icmt in cmtfile.readlines():
                cmtline = icmt.strip().split()
                if(cmtline[0] == "depth:"):
                    dep = float(cmtline[1])
                    if(depthrange[idep][0]=='d'):
                        dep_per = dep + float(depthrange[idep][1:])
                    elif(depthrange[idep][0]=='u'):
                        dep_per = dep - float(depthrange[idep][1:])
                    else:
                        dep_per = dep
                    cmt_per.writelines("depth:          "+str(dep_per)+"\n")
                else:
                    cmt_per.writelines(icmt)
            # end writing new cmts with depth perturb
        return 0
    else:
        print("not implemented per types other than depth")
        return -1
    
def init_structure(base, cmtfiles, stafiles, ref, evtnm):
    """
    base: base directory
    ref: where the specfem3d code locates
    cmtfiles: the directory to store cmtfiles
    stafiles: the directory to store station files
    """
    # base dir
    sh.mkdir("-p", base)
    # work dir
    sh.mkdir("-p", join(base, "work"))

    # get work directories' list
    dirs_raw = glob(join(cmtfiles, "*"))
    dirs = [i.split("/")[-1] for i in dirs_raw]
    
    # mkdir of the working directories
    for dir in dirs:
        sh.mkdir("-p", join(base, "work", dir))

    # ln the bin and DATABASE to work directories
    for dir in dirs:
        sh.ln("-s", join(ref, "bin"), join(base, "work", dir, "bin"))
        sh.ln("-s", join(ref, "DATABASES_MPI"), join(base, "work", dir, "DATABASES_MPI"))

    # mkdir DATA and OUTPUT_FILES in work directory
    for dir in dirs:
        sh.mkdir(join(base, "work", dir, "DATA"))
        sh.mkdir(join(base, "work", dir, "OUTPUT_FILES"))

    # cp and ln files in DATA
    toln = ["cemRequest", "crust1.0", "crust2.0",
            "crustmap", "epcrust", "eucrust-07", "GLL", "heterogen", 
            "Lebedev_sea99", "Montagner_model", "old", "PPM", "QRFSI12", 
            "s20rts", "s362ani", "s40rts", "Simons_model", "topo_bathy", "Zhao_JP_model"]
    for dir in dirs:
        sh.cp(join(cmtfiles, dir), join(
            base, "work", dir, "DATA", "CMTSOLUTION"))
        sh.cp(join(ref, "DATA", "Par_file"),
              join(base, "work", dir, "DATA", "Par_file"))
        sh.cp(join(stafiles, "STATIONS_"+evtnm),
              join(base, "work", dir, "DATA", "STATIONS"))
        for lnfile in toln:
            sh.ln("-s", join(ref, "DATA", lnfile),
                  join(base, "work", dir, "DATA", lnfile))

        # ln in workfiles
        toln_work = ["utils"]
        for lnfile in toln_work:
            sh.ln("-s", join(ref, lnfile),
                join(base, "work", dir, lnfile))

    # cp values from mesher and addressing in OUTPUT_FILES to WORK dirs
    for dir in dirs:
        sh.cp(join(ref, "OUTPUT_FILES", "addressing.txt"),
              join(base, "work", dir, "OUTPUT_FILES"))
        sh.cp(join(ref, "OUTPUT_FILES", "values_from_mesher.h"),
              join(base, "work", dir, "OUTPUT_FILES"))
    
    print("structure set up")
    return 0

def get_dirs(base):
    all_dirs = glob(join(base, "work", "*"))
    curdir = os.path.abspath(".")
    for idir in range(0,len(all_dirs)):
        all_dirs[idir] = join(curdir, all_dirs[idir])
    
    #all_dirs.remove("dep_pert_search/work/CMTSOLUTION_00")   # no need to calc '00', already calc'd
    return all_dirs   # right now in the test just use all the perturbations


def get_scripts(thedirs,N_each,N_iter,N_total,nproc):
    """
    input: 
    thedirs: directories for run, each event
    N_each: number of simulations at each iteration ###N_each*nproc <= 2760###
    N_iter: iterations
    N_total: total number of simulations
    nproc: cores per simulation
    """
    result = ""
    curdir = os.path.abspath(".")
    # for xspecfem3D
    result += f"echo 'start xspecfem3D'; "
    for iiter in range(N_iter):
        result += f"echo 'start iteration {iiter}'; "
        # to see if it is the last iter
        if(iiter == N_iter-1):
            n_remain = N_total - N_each * iiter
            for ieach in range(n_remain):
                # ievent
                ievent = iiter * N_each + ieach
                ievent_dir = thedirs[ievent]
                result += f"cd {ievent_dir}; "
                # if n_remain-1
                if(ieach == n_remain-1):
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D; "
                else:
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D & "
                # cd
                result += f"cd {curdir}; "
        else:  # normal iters
            for ieach in range(N_each):
                # ievent
                ievent = iiter*N_each+ieach
                ievent_dir = thedirs[ievent]
                # cd
                result += f"cd {ievent_dir}; "
                # if N_each-1
                if(ieach == N_each-1):
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D; "
                else:
                    inc = ieach*nproc
                    result += f"ibrun -n {nproc} -o {inc} ./bin/xspecfem3D & "
                # cd
                result += f"cd {curdir}; "
        result += f"wait; "
        result += f"echo 'end iteration {iiter}'; "

    return result


def submit_job(thecommand,N_each,nproc,esti_time):
    ## please change the time and the account accordingly
    ## safe time is 2*(simulation time for one event)*(N_iter)
    s = Slurm("sync", {"nodes": math.ceil(N_each*nproc/48), "ntasks": N_each*nproc,
                       "partition": 'skx-normal', "time": esti_time, "account": 'TG-EAR140030',
                      "mail-user": 'zhouton3@msu.edu', "mail-type": 'all'})
    s.run(thecommand)


In [4]:
event_list_file = "evtlst.txt"
base_dir = "/work/05880/tg851792/stampede2/WUS/CMT_inversion_test"
model_dir = '/work/05880/tg851792/stampede2/WUS/new_region_us2016hybrid'
nsimu = 0
nper = {}
cal_dirs = []

for ievt in open(event_list_file,'r').readlines():
    evtnm = ievt.strip()
    if(not os.path.exists("cmt3d_syn_"+evtnm)):
        sh.mkdir("p", "cmt3d_syn_"+evtnm)
    
    os.chdir("cmt3d_syn_"+evtnm)
    curdir = os.path.abspath(".")
    sh.cp(join(model_dir, "DATA", "CMTSOLUTION_"+evtnm),
              join(curdir, "CMTSOLUTION"))
    dep,_,_,_,_,_,_,_,_ = read_cmt("CMTSOLUTION")
    if(dep<6):
        dep_per = ['d8','d6','d4','d2','00','u2','u4']
    elif(dep<8):
        dep_per = ['d8','d6','d4','d2','00','u2','u4','u6']
    elif(dep<10):
        dep_per = ['d8','d6','d4','d2','00','u2','u4','u6','u8']
    elif(dep<15):
        dep_per = ['d12','d9','d6','d3','00','u3','u6','u9','u12']
    else:
        dep_per = ['d12','d9','d6','d3','00','u3','u6','u9','u12','u15']

    cmtsolution_per("CMTSOLUTION",'depth','./cmt_dep_per_'+evtnm,dep_per)  

    #init_structure('dep_pert_search','./cmt_dep_per_'+evtnm,model_dir+"/DATA/STAs",
    #               model_dir,evtnm)
    
    print("setting "+evtnm+" structure")
    nper[evtnm] = len(dep_per)
    nsimu += len(dep_per)
    
    cal_dirs += get_dirs("dep_pert_search")      # get run directories
    os.chdir("..")  # go to the origin dir
    
    
N_total = 12     # number of total simulations
N_each = 6      # number of simulations in each batch, should be 3x to fully utilize the node
N_iter = int(N_total/N_each)+1      # number of batches
nproc = 256     # number of cores in one single simulation

print(cal_dirs)

thecommand = get_scripts(cal_dirs[0:12],N_each,N_iter,N_total,nproc)
print(thecommand)
submit_job(thecommand,N_each,nproc,"1:40:00")    
#thecommand = get_scripts(cal_dirs[12:24],N_each,N_iter,N_total,nproc)
#print(thecommand)
#submit_job(thecommand,N_each,nproc,"1:40:00") 
#thecommand = get_scripts(cal_dirs[24:36],N_each,N_iter,N_total,nproc)
#print(thecommand)
#submit_job(thecommand,N_each,nproc,"1:40:00") 
#thecommand = get_scripts(cal_dirs[36:48],N_each,N_iter,N_total,nproc)
#print(thecommand)
#submit_job(thecommand,N_each,nproc,"1:40:00") 
    

setting 201501282109A structure
setting 201509130814A structure
setting 201509241349A structure
setting 201609031202A structure
setting 201612280913A structure
['/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d3', '/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d12', '/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u12', '/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u15', '/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u3', '/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d9', '/work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_00', '/work/05880/tg85

In [7]:
thecommand = get_scripts(cal_dirs[0:3],3,1,3,256)
print(thecommand)
submit_job(thecommand,3,nproc,"0:50:00")   
thecommand = get_scripts(cal_dirs[3:6],3,1,3,256)
print(thecommand)
submit_job(thecommand,3,nproc,"0:50:00")   
thecommand = get_scripts(cal_dirs[6:10],4,1,4,256)
print(thecommand)
submit_job(thecommand,4,nproc,"0:50:00")   

echo 'start xspecfem3D'; echo 'start iteration 0'; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d3; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d12; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u12; ibrun -n 256 -o 512 ./bin/xspecfem3D; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; wait; echo 'end iteration 0'; 


submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login2)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3248910'


echo 'start xspecfem3D'; echo 'start iteration 0'; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u15; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u3; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d9; ibrun -n 256 -o 512 ./bin/xspecfem3D; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; wait; echo 'end iteration 0'; 


submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login2)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3248913'


echo 'start xspecfem3D'; echo 'start iteration 0'; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_00; ibrun -n 256 -o 0 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u6; ibrun -n 256 -o 256 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_d6; ibrun -n 256 -o 512 ./bin/xspecfem3D & cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test/cmt3d_syn_201501282109A/dep_pert_search/work/CMTSOLUTION_u9; ibrun -n 256 -o 768 ./bin/xspecfem3D; cd /work/05880/tg851792/stampede2/WUS/CMT_inversion_test; wait; echo 'end iteration 0'; 


submitted: b'-----------------------------------------------------------------\n          Welcome to the Stampede2 Supercomputer                 \n-----------------------------------------------------------------\n\nNo reservation for this job\n--> Verifying valid submit host (login2)...OK\n--> Verifying valid jobname...OK\n--> Enforcing max jobs per user...OK\n--> Verifying availability of your home dir (/home1/05880/tg851792)...OK\n--> Verifying availability of your work dir (/work/05880/tg851792/stampede2)...OK\n--> Verifying availability of your scratch dir (/scratch/05880/tg851792)...OK\n--> Verifying valid ssh keys...OK\n--> Verifying access to desired queue (skx-normal)...OK\n--> Verifying job request is within current queue limits...OK\n--> Checking available allocation (TG-EAR140030)...OK\nSubmitted batch job 3248914'


In [3]:
a = ['a','v',3]
b = [4,5,6]
b+=(a)
print(b)
print(glob("*"))
os.chdir("CMT_inversion_test")
print(nsimu)
print(len(cal_dirs))

[4, 5, 6, 'a', 'v', 3]
['pre_proc_data.py', 'grid_search_manual', 'cmt3d_syn_201501282109A', 'scripts', 'spaceweight-master', 'SRC_CORRECT_INITIALTIME_MOMENT', 'Run_forward.ipynb', 'cmt3d_syn_200609101456A', 'inverse_problem_for_source', 'cmt3d_syn_200908031800A', 'cmt3d_syn_200804180937A', 'evtlst.txt', 'cmt3d_syn_201509241349A', 'setup_run_structure_for_depth_gridsearch.ipynb', 'pyCMT3D', 'cmt3d_syn_201609031202A', 'slurm-scripts', 'cmt3d_syn_200802211416A', 'SRC_GRIDSEARCH_INITIALTIME_MOMENT', '201006150427A', 'do_grid_search.ipynb', 'cmt3d_syn_200711262156A', 'cmt3d_syn_201612280913A', '201709100947A', 'logs', 'cmt3d_syn_201509130814A', 'cmt3d_syn_200803151444A', 'cmt3d_syn_201006150427A']


FileNotFoundError: [Errno 2] No such file or directory: 'CMT_inversion_test'